## Generating a Spotify Playlist

<a href="https://wfseaton.github.io/TheDigitalFrontier/">Home Page</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/data_preparation.html"><b>Data Preparation</b></a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/data_exploration.html">Data Exploration</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/dimensionality_reduction.html">Dimensionality Reduction</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/clustering_techniques.html">Clustering Techniques</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/playlist_generation.html">Playlist Generation</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/conclusion.html">Conclusion</a> - 
<a href="https://wfseaton.github.io/TheDigitalFrontier/authors_gift.html">Authors' Gift</a>

-------------------------------------------------------------------------------------------------------------------

# Preparing and Enriching the Million Playlist Dataset

In [1]:
import time
import math
import pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from os import listdir
from os.path import isfile, join
sns.set()

from itertools import chain

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

client_id = ""
client_secret = ""

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Data Source

We began our work with the Million Playlist Dataset. This data set was prepared as part of the [RecSys Challenge 2018](https://recsys-challenge.spotify.com/) organized by Spotify, University of Massachusetts (Amherst), and Johannes Kepler University (Linz).

The raw data includes:

- 1,000 CSV files, totaling 11.63 GB
- Each CSV file has 1,000 playlists, each with a collection of approximately 50-200 songs
- ~65,000 songs in each CSV file, ~65,000,000 total observations (including duplicates) 

The initial provided data features include:

- **Playlist Number:** integer value ranging from 0 to 999 within each CSV file
- **Track Position:** integer value starting from 0 indicating the position of the song in the playlist
- **Track Name:** text value
- **Track URI:** alphanumeric identifier from Spotify
- **Artist Name:** text value
- **Artist URI:** alphanumeric identifier from Spotify
- **Album Name:** text value
- **Album URI:** alphanumeric identifier from Spotify
- **Duration:** integer value in milliseconds

Here is a sample of the provided data.

In [4]:
# display a sample CSV file
df = pd.read_csv('../data/Songs/songs285.csv')
print("Shape of data in CSV file:", df.shape)
display(df.head())

Shape of data in CSV file: (64928, 9)


,pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Deftones,spotify:track:4rEGJ9KirDlKiOHxqVwcVg,spotify:artist:6Ghvu1VvMGScGpOUJBAHNH,Sextape,spotify:album:4RQnFSkkZlA65Xxchhnaha,241533,Diamond Eyes
1,0,1,Muse,spotify:track:0It6VJoMAare1zdV2wxqZq,spotify:artist:12Chz98pHFMPJEknJQMWvI,Undisclosed Desires,spotify:album:0eFHYz8NmK75zSplL5qlfM,235000,The Resistance
2,0,2,Pearl Jam,spotify:track:0LBmvPJYmtEJ7kkWvc3kbT,spotify:artist:1w5Kfo2jwwIPruYS2UWh56,Oceans,spotify:album:5B4PYA7wNN4WdEXdIJu58a,161893,Ten
3,0,3,My Chemical Romance,spotify:track:0uukw2CgEIApv4IWAjXrBC,spotify:artist:7FBcuc1gsnv6Y1nwFtNRCb,Dead!,spotify:album:0FZK97MXMm5mUQ8mtudjuK,195520,The Black Parade
4,0,4,Red Hot Chili Peppers,spotify:track:1iFIZUVDBCCkWe705FLXto,spotify:artist:0L8ExT028jH3ddEcZwqJJ5,Dosed,spotify:album:6deiaArbeoqp1xPEGdEKp1,311866,By The Way


## Data Structuring

For songs that appear in multiple playlists, data in each of the columns are repeated. This produces a total of 11.63 GB of data, a significant computational challenge. A reasonable first step to slim down the size of the dataset without losing information or fidelity is to parse through all the files to create a reference table of all songs and their metadata. Each playlist can then be stored as a simple named object, where the name is the ID of the playlist and its value a vector of song IDs.

From this exercise, we output two data frames:

- **Songs table:** Pandas dataframe with all songs as rows and all data from individual CSV files as columns
- **Playlists series:** Pandas series with playlist IDs as indices and vectors of song IDs as items

Running this over the entire dataset has a run-time of 2.5 hours. This method of storing songs in a dedicated table reduces 65 million song observations to just 2.5 million unique songs. To make this computationally tractable, we leveraged Pandas data frames and the fact that their indices, if sorted and maintained properly, leverage hash tables for quick lookups. This is in contrast to using a base Python method like loops or list comprehension, which would require searching the full table for the song each time. This reduced the overall data scale from 11.63 GB to just 0.42 GB for the songs master table and 0.54 GB for the playlist vectors, a total that is less than 10% the original size with no information loss. Our efforts in complexity reduction enabled us to perform our modeling at a significantly larger scale and use more data to generate better recommendations.

In [1]:
# Looping over 200,000 random playlists to fill out a master Pandas dataframe for songs and a Pandas series for playlists

start_time = time.time()
loop_start = time.time()

# List of all files
all_files = listdir('../data/Songs')
# 200,000 playlists is plenty
all_files = all_files[0:200]

# Load first file to get columns (standard across all)
df = pd.read_csv('../data/Songs/' + all_files[0])

# Master DataFrame of all unique songs included across all playlists
#songs = pd.DataFrame(columns = list(df.columns)[2:])
songs = pd.DataFrame()

# Master Series of playlists and the songs included in each
playlists = pd.Series()

# Aggregator functions to limit to one row per song and count occurrences across playlists
a1 = dict()
for key in df.columns[2:]:
    a1[key] = 'first'
a1['track_uri'] = 'count'

# Aggregator to consolidate into sum of songs across playlists
a2= dict()
for key in df.columns[2:]:
    a2[key] = 'first'
del a2['track_uri']
a2['count'] = 'sum'

# Loop over each file to extract data
for i, file in enumerate(all_files):
    # split on "." to split into "filename" and "csv"
    # Then select "filename" and ditch the first five letters "songs"
    filenum = file.split(".")[0][5:]
    
    # Load file and store in temporary dataframe
    fdf = pd.read_csv('../data/Songs/' + file)
    
    # --- SONGS IN FILE ---
    fdf_counts = fdf.iloc[:, 2:]
    fdf_counts = fdf_counts.groupby('track_uri').agg(a1)
    fdf_counts.rename(columns = {'track_uri': 'count'}, inplace = True)
    
    # Add to df of unique songs, update counters, and remove duplicates
    songs = songs.append(fdf_counts)
    
    # -- SONGS IN EACH PLAYLIST --
    # Songs included in every playlist (ordered) in file
    # For each playlist, get list of track_uri's (unique identifiers)
    songs_in_playlist = fdf.groupby('pid')['track_uri'].unique()

    # Update index to be not the pid in file (id), but a combination of them
    #songs_in_playlist.index = [filenum + '_' + str(pid) for pid in songs_in_playlist.index.values]
    songs_in_playlist.index = list(map(lambda x: filenum + '_' + str(x), songs_in_playlist.index.values))
    
    # Add playlists to master Series of all playlists
    playlists = playlists.append(songs_in_playlist)
    
    # Every 50 files, consolidate the songs table so it doesn't grow too big
    if (i+1)%25 == 0: 
        print('{}/{} -- {} s'.format(i+1, len(all_files), time.time() - loop_start))
        loop_start = time.time()
        songs = songs.groupby('track_uri').agg(a2, sort = True)
        print('   Consolidation: {} s'.format(time.time() - loop_start))
    
print("--- %s seconds ---" % (time.time() - start_time))

In [2]:
# Looping over all files to fill out a master Pandas dataframe for songs and a Pandas series for playlists

start_time = time.time()
loop_start = time.time()

# List of all files
all_files = listdir('../data/Songs')
# Limit to some of the playlists
#all_files = all_files[0:200]

# Load first file to get columns (standard across all)
df = pd.read_csv('../data/Songs/' + all_files[0])

# Master DataFrame of all unique songs included across all playlists
#songs = pd.DataFrame(columns = list(df.columns)[2:])
songs = pd.DataFrame()

# Master Series of playlists and the songs included in each
playlists = pd.Series()

# Aggregator functions to limit to one row per song and count occurrences across playlists
a1 = dict()
for key in df.columns[2:]:
    a1[key] = 'first'
a1['track_uri'] = 'count'

# Aggregator to consolidate into sum of songs across playlists
a2= dict()
for key in df.columns[2:]:
    a2[key] = 'first'
del a2['track_uri']
a2['count'] = 'sum'

# Loop over each file to extract data
for i, file in enumerate(all_files):
    # split on "." to split into "filename" and "csv"
    # Then select "filename" and ditch the first five letters "songs"
    filenum = file.split(".")[0][5:]
    
    # Load file and store in temporary dataframe
    fdf = pd.read_csv('../data/Songs/' + file)
    
    # --- SONGS IN FILE ---
    fdf_counts = fdf.iloc[:, 2:]
    fdf_counts = fdf_counts.groupby('track_uri').agg(a1)
    fdf_counts.rename(columns = {'track_uri': 'count'}, inplace = True)
    
    # Add to df of unique songs, update counters, and remove duplicates
    songs = songs.append(fdf_counts)
    
    # -- SONGS IN EACH PLAYLIST --
    # Songs included in every playlist (ordered) in file
    # For each playlist, get list of track_uri's (unique identifiers)
    songs_in_playlist = fdf.groupby('pid')['track_uri'].unique()

    # Update index to be not the pid in file (id), but a combination of them
    #songs_in_playlist.index = [filenum + '_' + str(pid) for pid in songs_in_playlist.index.values]
    songs_in_playlist.index = list(map(lambda x: filenum + '_' + str(x), songs_in_playlist.index.values))
    
    # Add playlists to master Series of all playlists
    playlists = playlists.append(songs_in_playlist)
    
    # Every 50 files, consolidate the songs table so it doesn't grow too big
    if (i+1)%25 == 0: 
        print('{}/{} -- {} s'.format(i+1, len(all_files), time.time() - loop_start))
        loop_start = time.time()
        songs = songs.groupby('track_uri').agg(a2, sort = True)
        print('   Consolidation: {} s'.format(time.time() - loop_start))
    
print("--- %s seconds ---" % (time.time() - start_time))

Once we have our master dataframe with all unique songs, we can assign an ID to each song, which we do as a new column at the end of the below dataframe labeled `song_id`.

In [18]:
# Do a final consolidation and add song ID to table
start_time = time.time()
songs_counts = songs.groupby('track_uri').agg(a2)
songs_counts['song_id'] = np.arange(len(songs_counts))
print("--- %s seconds ---" % (time.time() - start_time))

print(songs_counts.shape)
display(songs_counts.head())

--- 7.567456960678101 seconds ---
(1003760, 8)


,artist_name,artist_uri,track_name,album_uri,duration_ms,album_name,count,song_id
track_uri,,,,,,,,
spotify:track:0002yNGLtYSYtc0X6ZnFvp,Sidney Bechet's Blue Note Jazzmen,spotify:artist:2XouUSO0EAJ9gMMoHiXqMt,Muskrat Ramble,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,220293,Jazz Classics,1,0
spotify:track:00039MgrmLoIzSpuYKurn9,Zach Farlow,spotify:artist:2jTojc4rAsOMx6200a8Ah1,Thas What I Do,spotify:album:0UHfgx3ITlxePDXLaN5Y6x,222727,The Great Escape 2,2,1
spotify:track:0006Rv1e2Xfh6QooyKJqKS,Two Steps from Hell,spotify:artist:2qvP9yerCZCS0U1gZU8wYp,Nightwood,spotify:album:1BD29pKydSXe1EsHFj0GrQ,189638,Colin Frake On Fire Mountain,4,2
spotify:track:0007AYhg2UQbEm88mxu7js,Little Simz,spotify:artist:6eXZu6O7nAUA5z6vLV8NKI,Mandarin Oranges Part 2,spotify:album:32RJzqlapfiU0fr2l4SSW9,198000,E.D.G.E,1,3
spotify:track:0009mEWM7HILVo4VZYtqwc,Slam,spotify:artist:0Y0Kj7BOR5DM0UevuY7IvO,Movement,spotify:album:62VkRE2ucNvZDnYMCsnNDh,447534,Movement,1,4


With our generated `song_id`, we replace the `track_uri` in each playlist and switch `song_id` to become the new index column. This allows us to make faster lookups using `song_id` in our future work.

In [61]:
# Replace playlist track_uri with song_id

start_time = time.time()
loop_start = time.time()

playlists_songids = pd.Series(index = playlists.index)
playlists_songids = playlists_songids.astype(object)

i = 0
for ind, row in playlists.items():
    songids = np.array(songs_counts.loc[row, 'song_id'], 'int')
    playlists_songids.loc[str(ind)] = songids
    
    i += 1
    if i % (len(playlists)/20) == 0 == 0: 
        print('{}/{} -- {} s'.format(i, int(len(playlists)), time.time() - loop_start))
        loop_start = time.time()
    
print("--- %s seconds ---" % (time.time() - start_time))

print(playlists_songids.shape)
print(playlists_songids.head())

10000/200000 -- 38.7440550327301 s
20000/200000 -- 43.11882281303406 s
30000/200000 -- 44.18085217475891 s
40000/200000 -- 48.200636863708496 s
50000/200000 -- 52.817174196243286 s
60000/200000 -- 50.61113619804382 s
70000/200000 -- 57.99031400680542 s
80000/200000 -- 64.7493839263916 s
90000/200000 -- 67.53792810440063 s
100000/200000 -- 67.3009626865387 s
110000/200000 -- 68.4447910785675 s
120000/200000 -- 71.28671312332153 s
130000/200000 -- 72.44740080833435 s
140000/200000 -- 77.3995201587677 s
150000/200000 -- 80.82779884338379 s
160000/200000 -- 358.95413088798523 s
170000/200000 -- 90.39383912086487 s
180000/200000 -- 89.62128067016602 s
190000/200000 -- 100.55844020843506 s
200000/200000 -- 97.31215310096741 s
--- 1642.5336339473724 seconds ---
(200000,)
284_0    [340039, 125250, 881533, 653897, 49614, 356319...
284_1    [738782, 7646, 142078, 900881, 533258, 429837,...
284_2    [552361, 135177, 507876, 865927, 638474, 55164...
284_3    [214695, 27387, 700562, 448130, 1000188

In [62]:
# Change songs table to have song_id as index and track_uri as column
# We will be doing lookups on song_id while running

songs_counts_id = songs_counts.copy()
songs_counts_id['track_uri'] = songs_counts_id.index.values
songs_counts_id.set_index('song_id', inplace = True)
songs_counts_id.head()

,artist_name,artist_uri,track_name,album_uri,duration_ms,album_name,count,track_uri
song_id,,,,,,,,
0,Sidney Bechet's Blue Note Jazzmen,spotify:artist:2XouUSO0EAJ9gMMoHiXqMt,Muskrat Ramble,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,220293,Jazz Classics,1,spotify:track:0002yNGLtYSYtc0X6ZnFvp
1,Zach Farlow,spotify:artist:2jTojc4rAsOMx6200a8Ah1,Thas What I Do,spotify:album:0UHfgx3ITlxePDXLaN5Y6x,222727,The Great Escape 2,2,spotify:track:00039MgrmLoIzSpuYKurn9
2,Two Steps from Hell,spotify:artist:2qvP9yerCZCS0U1gZU8wYp,Nightwood,spotify:album:1BD29pKydSXe1EsHFj0GrQ,189638,Colin Frake On Fire Mountain,4,spotify:track:0006Rv1e2Xfh6QooyKJqKS
3,Little Simz,spotify:artist:6eXZu6O7nAUA5z6vLV8NKI,Mandarin Oranges Part 2,spotify:album:32RJzqlapfiU0fr2l4SSW9,198000,E.D.G.E,1,spotify:track:0007AYhg2UQbEm88mxu7js
4,Slam,spotify:artist:0Y0Kj7BOR5DM0UevuY7IvO,Movement,spotify:album:62VkRE2ucNvZDnYMCsnNDh,447534,Movement,1,spotify:track:0009mEWM7HILVo4VZYtqwc


In [ ]:
# Save data as CSV and PKL files

songs_counts_id.to_csv('../data/songs_counts_'+str(len(all_files))+'.csv')
songs_counts_id.to_pickle('../data/songs_counts_'+str(len(all_files))+'.pkl')

playlists_songids.to_csv('../data/playlists_song_ids_'+str(len(all_files))+'.csv', header = False)
playlists_songids.to_pickle('../data/playlists_song_ids_'+str(len(all_files))+'.pkl')

## Enriching a Song's Musical Features

Our recommendation hypothesis is that song features provide a data-based way of determining similarity and thus good matches to our seed songs. We leverage Spotify's open API to retrieve these musical features and add it into our dataset using Spotipy, a lightweight Python library that allows us to authenticate to Spotify and easily query features on the song, artist, and album. 

To keep Spotify API requests reasonable, we randomly selected 200,000 playlists (out of the total 1,000,000). Across these 200,000 playlists, we have 1,003,760 unique songs. We pull the data listed below for songs, artists, and albums from the Spotify API. Descriptions here are directly from [Spotify API reference documentation](https://developer.spotify.com/documentation/web-api/reference/).

#### Song features
- **acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
- **danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
- **energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- **instrumentalness:** Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- **key:** The key the track is in. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on.
- **liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- **loudness:** The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
- **mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
- **speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- **tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- **time_signature:** An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).
- **valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

#### Artist features
- **artist_genres:** A list of the genres the artist is associated with. For example: "Prog Rock" , "Post-Grunge". (If not yet classified, the array is empty.)
- **artist_popularity:** The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks.

#### Album features
- **album_genres:** A list of the genres used to classify the album. For example: "Prog Rock" , "Post-Grunge". (If not yet classified, the array is empty.)
- **album_popularity:** The popularity of the album. The value will be between 0 and 100, with 100 being the most popular. The popularity is calculated from the popularity of the album’s individual tracks.
- **release_date:** The date the album was first released. 

In [ ]:
# Enrich data with new track features from Spotify API

# read songs table
df = pd.read_pickle('../data/songs_counts_200.pkl')

# define batches
batch_size = 100
num_batches = math.ceil(len(df)/batch_size)

# initialize list to save API calls
track_features = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    track_list = list(df['track_uri'][start_point:end_point])
    track_features.extend(sp.audio_features(track_list))

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()

# convert to dataframe
track_features_df = pd.DataFrame(track_features)

# save data
track_features_df.to_csv('../data/track_features.csv')
track_features_df.to_pickle('../data/track_features.pkl')

In [ ]:
# Enrich data with new artist features from Spotify API

# identify unique artists
unique_artists = list(df['artist_uri'].unique())

# define batches
batch_size = 50
num_batches = math.ceil(len(unique_artists)/batch_size)

# initialize list to save API calls
artist_info = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    artist_list = unique_artists[start_point:end_point]
    artist_info.extend(sp.artists(artist_list)['artists'])

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()

# convert to dataframe
artist_info_df = pd.DataFrame(artist_info)

# save data
artist_info_df.to_csv('../data/artist_info.csv')
artist_info_df.to_pickle('../data/artist_info.pkl')

In [ ]:
# Enrich data with new album features from Spotify API

# identify unique albums
unique_albums = list(df['album_uri'].unique())

# define batches
batch_size = 20
num_batches = math.ceil(len(unique_albums)/batch_size)

# initialize list to save API calls
album_info = []

start_time = time.time()

# looping through the batches
for i in range(num_batches):
    
    
    # define start and end of the batch
    start_point = i*batch_size
    end_point = min(start_point + batch_size, len(df))
    
    # API call
    album_list = unique_albums[start_point:end_point]
    album_info.extend(sp.albums(album_list)['albums'])

    if i%100 == 0:
        print('{}/{}, {}s'.format(i, num_batches, time.time()-start_time))
        start_time = time.time()
        
# eliminates none values
album_info = [i for i in album_info if i is not None]

# convert to dataframe
album_info_df = pd.DataFrame(album_info)

# save data
album_info_df.to_csv('../data/album_info.csv')
album_info_df.to_pickle('../data/album_info.pkl')

After pulling data from the Spotify API, we have a dataset of 999,950 unique songs and their associated metadata. We join our retrieved features back into the master tables to produce our final data frames below. We conclude by saving our transformed data into pickle files, which provides faster and more compact files for checkpoint saving.

#### Enriched Tracks Dataframe

In [130]:
# import data
track_columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','uri']
tracks = pd.read_csv('../data/track_features3.csv', usecols=track_columns)

# rename columns for clarity
tracks = tracks.rename(columns={'uri': 'track_uri'})
tracks = tracks.drop_duplicates()

tracks

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_uri,time_signature
0,0.455,0.623,8,-11.572,1,0.0523,0.797000,0.903000,0.6340,0.9510,182.345,spotify:track:0002yNGLtYSYtc0X6ZnFvp,4
1,0.742,0.753,1,-5.632,1,0.0364,0.017800,0.000000,0.1330,0.2630,132.064,spotify:track:00039MgrmLoIzSpuYKurn9,4
2,0.295,0.498,2,-9.190,0,0.0301,0.795000,0.944000,0.1070,0.0445,89.048,spotify:track:0006Rv1e2Xfh6QooyKJqKS,3
3,0.648,0.598,7,-11.845,1,0.3260,0.164000,0.000046,0.1230,0.4000,138.883,spotify:track:0007AYhg2UQbEm88mxu7js,4
4,0.695,0.828,1,-6.818,1,0.0457,0.000007,0.851000,0.1090,0.0372,126.014,spotify:track:0009mEWM7HILVo4VZYtqwc,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003729,0.447,0.724,1,-6.398,0,0.0372,0.788000,0.202000,0.2420,0.9400,81.071,spotify:track:7zzptITgTKf4HpJM8ye47v,4
1003730,0.497,0.698,7,-2.558,1,0.0317,0.127000,0.000000,0.1160,0.5520,129.996,spotify:track:7zzpwV2lgKsLke68yFoZdp,4
1003731,0.314,0.359,0,-14.035,1,0.0378,0.745000,0.866000,0.6700,0.1030,131.911,spotify:track:7zzrUgpSu2MSZF4FecBN3D,4
1003732,0.769,0.644,11,-5.568,0,0.0997,0.242000,0.010200,0.0509,0.3960,103.014,spotify:track:7zzuTn6PnJ1DVfAiGsd4N0,4


#### Enriched Artists Dataframe

In [129]:
# import data
artist_columns = ['genres','popularity','uri']
artists = pd.read_csv('../data/artist_info1.csv', usecols=artist_columns)

# rename columns for clarity
artists = artists.rename(columns={'genres': 'artist_genres', 'popularity': 'artist_popularity', 'uri': 'artist_uri'})
artists = artists.drop_duplicates()

artists

,artist_genres,artist_popularity,artist_uri
0,[],18,spotify:artist:2XouUSO0EAJ9gMMoHiXqMt
1,[],27,spotify:artist:2jTojc4rAsOMx6200a8Ah1
2,"['epicore', 'scorecore', 'soundtrack', 'video ...",70,spotify:artist:2qvP9yerCZCS0U1gZU8wYp
3,"['alternative r&b', 'escape room', 'indie r&b'...",63,spotify:artist:6eXZu6O7nAUA5z6vLV8NKI
4,"['acid techno', 'minimal dub', 'minimal techno...",39,spotify:artist:0Y0Kj7BOR5DM0UevuY7IvO
...,...,...,...
149667,[],12,spotify:artist:2mRZOivAjsqp7VbLrjfL5g
149668,[],15,spotify:artist:4Cx5NofT2E2Ypu32HP0Mot
149669,"['dub reggae', 'uk dub']",15,spotify:artist:3RQFJaSRrhXQ2E9T8vZEe3
149670,[],17,spotify:artist:7iihQll6y9O8Iee7D1uEcb


#### Enriched Albums Dataframe

In [128]:
# import data
album_columns = ['genres','popularity','release_date','uri']

albums1 = pd.read_csv('../data/album_info1.csv', usecols=album_columns)
albums2 = pd.read_csv('../data/album_info2.csv', usecols=album_columns)
albums3 = pd.read_csv('../data/album_info3.csv', usecols=album_columns)
albums4 = pd.read_csv('../data/album_info4.csv', usecols=album_columns)
albums5 = pd.read_csv('../data/album_info5.csv', usecols=album_columns)
albums6 = pd.read_csv('../data/album_info6.csv', usecols=album_columns)
albums = pd.concat([albums1, albums2, albums3, albums4, albums5, albums6], axis=0, ignore_index=True)

# rename columns for clarity
albums = albums.rename(columns={'genres': 'album_genres', 'popularity': 'album_popularity', 'release_date': 'album_release_date', 'uri': 'album_uri'})
albums = albums.drop_duplicates()

albums

,album_genres,album_popularity,album_release_date,album_uri
0,[],37,1993-01-01,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY
1,[],0,2016-01-08,spotify:album:0UHfgx3ITlxePDXLaN5Y6x
2,[],41,2014-07-01,spotify:album:1BD29pKydSXe1EsHFj0GrQ
3,[],33,2014-10-03,spotify:album:32RJzqlapfiU0fr2l4SSW9
4,[],8,2013-08-26,spotify:album:62VkRE2ucNvZDnYMCsnNDh
...,...,...,...,...
375441,[],33,2017-08-25,spotify:album:6Fnj40x1kkxtHK3icGVsqg
375442,[],0,2014-09-16,spotify:album:5lkHAkEZsMj2bvcAzzrJYz
375443,[],25,2016-06-29,spotify:album:2OhSv0hAHVqo5zIDaUDcQA
375444,[],22,1994-09-09,spotify:album:4WmaRhaV8rs1GOk1GX26j5


#### Total and Complete Master Songs Data Frame

In [4]:
master = pd.read_pickle('../data/songs_counts_200.pkl')
master['song_id'] = master.index
master

In [132]:
master = master.merge(track_features, on='track_uri', suffixes=(None, '_tracks'))
master = master.merge(artists, on='artist_uri', suffixes=(None, '_artists'))
master = master.merge(albums, on='album_uri', suffixes=(None, '_albums'))
master = master.set_index('song_id')
master

,artist_name,artist_uri,track_name,album_uri,duration_ms,album_name,count,track_uri,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,artist_genres,artist_popularity,album_genres,album_popularity,album_release_date
song_id,,,,,,,,,,,,,,,,,,,,,
0,Sidney Bechet's Blue Note Jazzmen,spotify:artist:2XouUSO0EAJ9gMMoHiXqMt,Muskrat Ramble,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,220293,Jazz Classics,1,spotify:track:0002yNGLtYSYtc0X6ZnFvp,0.455,0.623,...,0.90300,0.6340,0.9510,182.345,4,[],18,[],37,1993-01-01
159583,Sidney Bechet,spotify:artist:1RsmXc1ZqW3WBs9iwxiSwk,Blue Horizon,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,264933,Jazz Classics,5,spotify:track:1EWPMNHfdVNJwBpG9BcxXB,0.327,0.372,...,0.83500,0.1530,0.3800,66.036,4,"['bebop', 'big band', 'cool jazz', 'dixieland'...",52,[],37,1993-01-01
271702,Sidney Bechet,spotify:artist:1RsmXc1ZqW3WBs9iwxiSwk,Blame It On The Blues - Alternate Take,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,175893,Jazz Classics,1,spotify:track:26N4Y48EjprAtvlY6yWZTA,0.574,0.606,...,0.94800,0.3490,0.9650,101.361,4,"['bebop', 'big band', 'cool jazz', 'dixieland'...",52,[],37,1993-01-01
445190,Sidney Bechet,spotify:artist:1RsmXc1ZqW3WBs9iwxiSwk,Summertime,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,251906,Jazz Classics,16,spotify:track:3RlJx8xwZEyToSuGrygilr,0.608,0.138,...,0.90800,0.0853,0.3180,83.124,4,"['bebop', 'big band', 'cool jazz', 'dixieland'...",52,[],37,1993-01-01
626275,Sidney Bechet,spotify:artist:1RsmXc1ZqW3WBs9iwxiSwk,Dear Old Southland,spotify:album:04hQBJ7YSuNnZ0nbuXNYbY,243693,Jazz Classics,1,spotify:track:4qwAa1rOm8iaegHzoM1b31,0.400,0.320,...,0.84200,0.1950,0.6130,86.186,4,"['bebop', 'big band', 'cool jazz', 'dixieland'...",52,[],37,1993-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003684,Royal Rizow,spotify:artist:5eXba1Axr3YJgg5j8Hn7a8,I'll Find a Way (feat. Ty Reynolds),spotify:album:3XWD2ACwxS3fnGGgu298eS,205346,I'll Find a Way (feat. Ty Reynolds),11,spotify:track:7zy1Ck7lyk9oruK1xozs7e,0.733,0.876,...,0.00000,0.0830,0.3980,104.947,4,[],11,[],13,2016-08-05
1003707,Moa Felicia,spotify:artist:2uB6FyYwjUENL072rdwu5B,Later That Night - Original Version,spotify:album:2ugoxa1ZsUKtbziD0SonBA,329032,Later That Night - Single,1,spotify:track:7zyYROflk5MgxcMjpNrCqT,0.804,0.513,...,0.75200,0.0686,0.0394,124.003,4,[],0,[],0,2013
1003733,The Commercial Hippies,spotify:artist:4Cx5NofT2E2Ypu32HP0Mot,The Antidote - Original Mix,spotify:album:2UnJeq5hGlBFCcnQZZ2Va9,511639,The Antidote,1,spotify:track:7zzGEjXJF5HYT5hghmGDXs,0.627,0.908,...,0.60300,0.2350,0.1660,140.007,4,[],15,[],8,2016-09-19


In [ ]:
master.to_csv('../data/master200.csv')
master.to_pickle('../data/master200.pkl')